In [2]:
!pip install pandas

Looking in indexes: https://artifacts.benteler.net/repository/pypi/simple


In [22]:
import re

description = "ATI Allegheny 304/304L - GW x 95% MSR > chart"
line = "304/304L $0.5169 $0.2160 $0.0000 $0.0000 $0.0000 $0.1013 $0.0000 $0.0000 $0.0000 $0.0000 $0.0002 $0.0552 $0.0000 $0.8896"

# Extract grade from description
match = re.search(r"(\d{3}(?:/\d{3}[A-Z]*)?)", description)
if match:
    grade = match.group(1)
    print(f"Grade found: {grade}")

# Extract dollar values from line
dollar_values = re.findall(r"\$\d+\.\d+", line)
print(f"Dollar values found: {dollar_values}")

# Print only the final value
if dollar_values:
    final_value = dollar_values[-1]
    print(f"Final value: {final_value}")


Grade found: 304/304L
Dollar values found: ['$0.5169', '$0.2160', '$0.0000', '$0.0000', '$0.0000', '$0.1013', '$0.0000', '$0.0000', '$0.0000', '$0.0000', '$0.0002', '$0.0552', '$0.0000', '$0.8896']
Final value: $0.8896


In [43]:
import re

description = "ATI Allegheny 304/304L - GW x 95% MSR > chart"
line = "304/304L $0.5169 $0.2160 $0.0000 $0.0000 $0.0000 $0.1013 $0.0000 $0.0000 $0.0000 $0.0000 $0.0002 $0.0552 $0.0000 $0.8896"

# Extract grade from description
match = re.search(r"(\d{3}(?:/\d{3}[A-Z]*)?)", description)
if match:
    grade = match.group(1)
    print(f"Grade found: {grade}")

# Extract dollar values from line
dollar_values = re.findall(r"\$\d+\.\d+", line)
print(f"Dollar values found: {dollar_values}")

# Convert to float and find the max value
if dollar_values:
    values_as_floats = [float(val.strip('$')) for val in dollar_values]
    max_value = max(values_as_floats)
    print(f"Max value: {max_value}")


Grade found: 304/304L
Dollar values found: ['$0.5169', '$0.2160', '$0.0000', '$0.0000', '$0.0000', '$0.1013', '$0.0000', '$0.0000', '$0.0000', '$0.0000', '$0.0002', '$0.0552', '$0.0000', '$0.8896']
Max value: 0.8896


In [12]:
import pandas as pd
import fitz  # PyMuPDF
import re

# === Load Excel ===
excel_path = "Sample_Surcharge_May.xlsx"
df = pd.read_excel(excel_path, engine='openpyxl')

# === Load ATI PDFs ===
pdf_greater_path = "ATI_Greater_May.pdf"
pdf_less_path = "ATI_Less_May.pdf"

def extract_pdf_data(pdf_path):
    """Extract the 'Total' column (14th dollar value) for each grade."""
    print(f"\nProcessing PDF: {pdf_path}")
    grade_to_rate = {}

    with fitz.open(pdf_path) as doc:
        for page in doc:
            lines = page.get_text("text").splitlines()
            for line in lines:
                if "$" not in line:
                    continue

                dollar_values = re.findall(r'\$([\d\.]+)', line)
                if len(dollar_values) < 14:
                    continue

                split_idx = line.find('$')
                grade = line[:split_idx].strip()

                try:
                    rate = float(dollar_values[13])
                    grade_to_rate[grade] = rate
                    print(f"  Grade: {grade} → Total Rate: {rate}")
                except ValueError:
                    continue

    return grade_to_rate

# === Extract Data from PDFs ===
print("\n=== Extracting from PDFs ===")
greater_data = extract_pdf_data(pdf_greater_path)
less_data = extract_pdf_data(pdf_less_path)

def extract_grade(description):
    clean_desc = description.upper()
    clean_desc = re.sub(r'\b(ATI|ALLEGHENY|GW|MSR|CHART)\b', '', clean_desc)
    clean_desc = re.sub(r'[^A-Z0-9/+-]', ' ', clean_desc)  # preserve grade symbols
    clean_desc = re.sub(r'\s+', ' ', clean_desc).strip()

    match = re.search(r'\b\d{3}[A-Z0-9/+-]*\b', clean_desc)
    return match.group(0) if match else None

def determine_chart_type(description):
    description = description.lower()
    if "greater" in description or ">" in description:
        return "greater"
    elif "less" in description or "<" in description:
        return "less"
    return None

def normalize_grade(g):
    return re.sub(r'\s+', '', g.lower())

# === Process Excel Rows ===
print("\n=== Matching Grades in Excel ===")
rates = []

for idx, row in df.iterrows():
    description = str(row["Description of Calculation"])
    chart_type = determine_chart_type(description)
    print(f"\nRow {idx} Description: {description}")
    grade = extract_grade(description)

    rate = None
    if chart_type and grade and re.search(r'ati|allegheny', description, re.IGNORECASE):
        data_source = greater_data if chart_type == "greater" else less_data
        print(f"Row {idx}: Grade={grade}, Chart={chart_type}")

        # Direct match
        if grade in data_source:
            rate = data_source[grade]
        else:
            # Try fuzzy/partial match
            grade_norm = normalize_grade(grade)
            matches = [
                g for g in data_source
                if grade_norm in normalize_grade(g) or normalize_grade(g) in grade_norm
            ]
            if matches:
                best_match = matches[0]
                rate = data_source[best_match]
                print(f"  → Fallback to '{best_match}' = {rate}")
            else:
                print("  ✘ No match found.")
    else:
        print(f"Row {idx}: Skipped — Chart Type: {chart_type}, Grade: {grade}, Contains ATI: {'ati' in description.lower()}")

    rates.append(rate)

df["Rates"] = rates
output_path = "Updated_Surcharge_May.xlsx"
df.to_excel(output_path, index=False)
print(f"\n✅ Done. Output saved to: {output_path}")



=== Extracting from PDFs ===

Processing PDF: ATI_Greater_May.pdf

Processing PDF: ATI_Less_May.pdf

=== Matching Grades in Excel ===

Row 0 Description: ATI Allefhney 309/309S GWxMSR >chart
Row 0: Grade=309/309S, Chart=greater
  ✘ No match found.

Row 1 Description: ATI Allegheny 304/304L - GW x 95% MSR > chart
Row 1: Grade=304/304L, Chart=greater
  ✘ No match found.

Row 2 Description: ATI Allegheny 304/304L - GW x MSR > chart
Row 2: Grade=304/304L, Chart=greater
  ✘ No match found.

Row 3 Description: ATI Allegheny 309 - GW x MSR < chart
Row 3: Grade=309, Chart=less
  ✘ No match found.

Row 4 Description: ATI Allegheny 309/309S - GW x 95% MSR > chart
Row 4: Grade=309/309S, Chart=greater
  ✘ No match found.

Row 5 Description: ATI Allegheny 309/309S - GW x MSR > chart
Row 5: Grade=309/309S, Chart=greater
  ✘ No match found.

Row 6 Description: ATI Allegheny 316Ti - GW x  MSR < chart
Row 6: Grade=316TI, Chart=less
  ✘ No match found.

Row 7 Description: ATI Allegheny 316Ti - GW x 95%

In [13]:
import pandas as pd
import fitz  # PyMuPDF
import re
import difflib  # for fuzzy matching

# === Load Excel ===
excel_path = "Sample_Surcharge_May.xlsx"
df = pd.read_excel(excel_path, engine='openpyxl')

# === Load ATI PDFs ===
pdf_greater_path = "ATI_Greater_May.pdf"
pdf_less_path = "ATI_Less_May.pdf"

def extract_pdf_data(pdf_path):
    """Extract the 'Total' column (14th dollar value) for each grade."""
    print(f"\nProcessing PDF: {pdf_path}")
    grade_to_rate = {}

    with fitz.open(pdf_path) as doc:
        for page in doc:
            lines = page.get_text("text").splitlines()
            for line in lines:
                if "$" not in line:
                    continue

                dollar_values = re.findall(r'\$([\d\.]+)', line)
                if len(dollar_values) < 14:
                    continue

                split_idx = line.find('$')
                grade = line[:split_idx].strip()

                try:
                    rate = float(dollar_values[13])
                    grade_to_rate[grade] = rate
                    print(f"  Grade: '{grade}' → Total Rate: {rate}")
                except ValueError:
                    continue

    return grade_to_rate

# === Extract Data from PDFs ===
print("\n=== Extracting from PDFs ===")
greater_data = extract_pdf_data(pdf_greater_path)
less_data = extract_pdf_data(pdf_less_path)

# Show all PDF grades for visual reference
print("\nGrades in GREATER chart:")
for g in greater_data:
    print(f" - {g}")
print("\nGrades in LESS chart:")
for g in less_data:
    print(f" - {g}")

def extract_grade(description):
    clean_desc = description.upper()
    clean_desc = re.sub(r'\b(ATI|ALLEGHENY|GW|MSR|CHART)\b', '', clean_desc)
    clean_desc = re.sub(r'[^A-Z0-9/+-]', ' ', clean_desc)
    clean_desc = re.sub(r'\s+', ' ', clean_desc).strip()

    match = re.search(r'\b\d{3}[A-Z0-9/+-]*\b', clean_desc)
    return match.group(0) if match else None

def determine_chart_type(description):
    description = description.lower()
    if "greater" in description or ">" in description:
        return "greater"
    elif "less" in description or "<" in description:
        return "less"
    return None

def normalize_grade(g):
    return re.sub(r'[^a-z0-9]', '', g.lower())  # remove all non-alphanum

# === Process Excel Rows ===
print("\n=== Matching Grades in Excel ===")
rates = []

for idx, row in df.iterrows():
    description = str(row["Description of Calculation"])
    chart_type = determine_chart_type(description)
    print(f"\nRow {idx} Description: {description}")
    grade = extract_grade(description)

    rate = None
    if chart_type and grade and re.search(r'ati|allegheny', description, re.IGNORECASE):
        data_source = greater_data if chart_type == "greater" else less_data
        print(f"Row {idx}: Grade='{grade}', Chart={chart_type}")

        norm_grade = normalize_grade(grade)

        # Normalize PDF grades for better matching
        normalized_pdf_grades = {normalize_grade(k): k for k in data_source}

        # Try exact normalized match
        if norm_grade in normalized_pdf_grades:
            actual_key = normalized_pdf_grades[norm_grade]
            rate = data_source[actual_key]
            print(f"  ✓ Exact normalized match to '{actual_key}' = {rate}")
        else:
            # Try fuzzy matching using difflib
            close_matches = difflib.get_close_matches(norm_grade, normalized_pdf_grades.keys(), n=1, cutoff=0.7)
            if close_matches:
                best_match_norm = close_matches[0]
                actual_key = normalized_pdf_grades[best_match_norm]
                rate = data_source[actual_key]
                print(f"  → Fuzzy match to '{actual_key}' = {rate}")
            else:
                print("  ✘ No match found.")
    else:
        print(f"Row {idx}: Skipped — Chart Type: {chart_type}, Grade: {grade}, Contains ATI: {'ati' in description.lower()}")

    rates.append(rate)

df["Rates"] = rates
output_path = "Updated_Surcharge_May.xlsx"
df.to_excel(output_path, index=False)
print(f"\n✅ Done. Output saved to: {output_path}")


=== Extracting from PDFs ===

Processing PDF: ATI_Greater_May.pdf

Processing PDF: ATI_Less_May.pdf

Grades in GREATER chart:

Grades in LESS chart:

=== Matching Grades in Excel ===

Row 0 Description: ATI Allefhney 309/309S GWxMSR >chart
Row 0: Grade='309/309S', Chart=greater
  ✘ No match found.

Row 1 Description: ATI Allegheny 304/304L - GW x 95% MSR > chart
Row 1: Grade='304/304L', Chart=greater
  ✘ No match found.

Row 2 Description: ATI Allegheny 304/304L - GW x MSR > chart
Row 2: Grade='304/304L', Chart=greater
  ✘ No match found.

Row 3 Description: ATI Allegheny 309 - GW x MSR < chart
Row 3: Grade='309', Chart=less
  ✘ No match found.

Row 4 Description: ATI Allegheny 309/309S - GW x 95% MSR > chart
Row 4: Grade='309/309S', Chart=greater
  ✘ No match found.

Row 5 Description: ATI Allegheny 309/309S - GW x MSR > chart
Row 5: Grade='309/309S', Chart=greater
  ✘ No match found.

Row 6 Description: ATI Allegheny 316Ti - GW x  MSR < chart
Row 6: Grade='316TI', Chart=less
  ✘ No 